# Task: implement a character RNN model to predict next characters

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
from fastai.column_data import *

PATH = Path('data/nlp')

class RNNLearner(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)

    def _get_crit(self, data): return F.nll_loss

    def summary(self): return model_summary(self.model, [torch.ones(3, dtype=torch.int64), torch.ones(3, dtype=torch.int64)])


class RNNModel(BasicModel):
    def get_layer_groups(self): return self.model

In [20]:
# text = open(PATH/'nietzsche.txt').read()
# print('corpus length:', len(text))

corpus length: 600893


In [21]:
# chars = sorted(list(set(text)))
# chars.insert(0,'\0') # padding chars
# n_char = len(chars)
# n_char
# ''.join(chars[0:])

85

'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆäæéë'

In [3]:
text = open(PATH/'KVH.txt').read()
print('corpus length:', len(text))

corpus length: 578177


In [4]:
text[0:500]

'Chương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang hai chiếc giường bên cạnh, thấy Quý ròm và nhỏ Hạnh vẫn còn ngủ say sưa, nó biếng nhác nằm ườn thêm một lát .\n\nỞ nhà, Tiểu Long không bao giờ như thế . Hễ mở mắt là nó ngồi bật dậy, tót xuống khỏi đi-văng . Cả anh Tuấn và anh Tú cũng vậy . Tác phong con nhà võ bao giờ cũng nhanh gọn lẹ làng .\n\nNhưng chiều nay chiếc giường nệm nhà Quý ròm đã níu lưng nó xuống . Đến chơi nhà Quý ròm, thỉnh thoảng Tiểu Long vẫn nằm trên chiếc giường này và lần nào'

In [5]:
chars = sorted(list(set(text)))
chars.insert(0,'\0') # padding chars
n_char = len(chars)
n_char

174

In [6]:
''.join(chars[0:])

'\x00\n !"%()+,-.0123456789:;?ABCDEGHIJKLMNOPQRSTUVXY_abcdefghiklmnopqrstuvwxy°ÀÁÂÊÌÍÐÒÔÚÝàáâãèéêìíòóôõùúýĂăĐđĩũƠơƯưẠạẢảấẦầẩẫậắằẳẵặẹẻẽẾếềỂểễệỉịọỎỏỐốỒồỔổỗộớỜờỞởỡợụỦủứỪừửữựỳỵỷỹ–’“”…'

In [7]:
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for i,c in enumerate(chars)}

# 3 chars models (hard-coded)

In [8]:
class ThreeCharsRNNAdd(nn.Module):
    def __init__(self,n_uchars,n_factors,n_hidden):
        super().__init__()
        self.emb = nn.Embedding(n_uchars,n_factors)
        self.w_in = nn.Linear(n_factors,n_hidden)
        self.w_hidden= nn.Linear(n_hidden,n_hidden)
        self.w_out = nn.Linear(n_hidden,n_uchars)
        
    def forward(self,c1,c2,c3):
        
        i1_h = F.relu(self.w_in(self.emb(c1))) #(bs,n_hidden)
        
        h = V(torch.zeros(i1_h.size()).cuda())
        h = h+i1_h # (bs,n_hidden)
        h = F.tanh(self.w_hidden(h)) #(bs,n_hidden)
        
        i2_h = F.relu(self.w_in(self.emb(c2)))
        h = F.tanh(self.w_hidden(h+i2_h))
        
        i3_h = F.relu(self.w_in(self.emb(c3)))
        h = F.tanh(self.w_hidden(h+i3_h))
        
        out = F.log_softmax(self.w_out(h)) # (bs,n_uchars)
        return out

## prepare dataset

In [9]:
cs=3
l = len(text)
print(l)
c1_idxs = [char2idx[c] for c in text[0:l-cs]]
c2_idxs = [char2idx[c] for c in text[1:l-(cs-1)]]
c3_idxs = [char2idx[c] for c in text[2:l-(cs-2)]]
c4_idxs = [char2idx[c] for c in text[3:l-(cs-3)]]

578177


In [10]:
''.join([idx2char[i] for i in c1_idxs[:50]])
''.join([idx2char[i] for i in c3_idxs[:50]])
''.join([idx2char[i] for i in c4_idxs[:50]])

'Chương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc san'

'ương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang '

'ơng 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang h'

In [11]:
x1 = np.array(c1_idxs)
x2 = np.array(c2_idxs)
x3 = np.array(c3_idxs)
y = np.array(c4_idxs)
print(x1.shape)
print(x3.shape)
print(y.shape)

(578174,)
(578174,)
(578174,)


## Fastai columnar data model

In [12]:
# Use columnar data model (with loaders) from fast.ai
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, is_reg=False,bs=512,shuffle=False)

In [13]:
# check if data is loaded correctly
temp = next(iter(md.trn_dl))
len(temp)
''.join([idx2char[i] for i in temp[0][:50]])
''.join([idx2char[i] for i in temp[1][:50]])
''.join([idx2char[i] for i in temp[2][:50]])
''.join([idx2char[i] for i in temp[3][:50]])

4

'Chương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc san'

'hương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang'

'ương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang '

'ơng 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang h'

In [14]:
len(md.trn_y)
''.join([idx2char[i] for i in md.trn_y[:50]])

578173

'ơng 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang h'

In [16]:
n_factor = n_char//2
model = ThreeCharsRNNAdd(n_char,n_factor,512).cuda()
model

ThreeCharsRNNAdd(
  (emb): Embedding(174, 87)
  (w_in): Linear(in_features=87, out_features=512, bias=True)
  (w_hidden): Linear(in_features=512, out_features=512, bias=True)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

## Low abstract fast.ai fit function

In [18]:
opt = optim.Adam(model.parameters(), 1e-2)

fit(model, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.085446   1.843928  
    1      2.135862   3.867795                                



[array([3.86779])]

In [19]:
set_lrs(opt, 1e-3)
fit(model, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                                
    0      1.668541   1.034816  



[array([1.03482])]

## Higher abstract fast.ai Learner class

In [98]:
model = RNNModel(to_gpu(model))

learner = RNNLearner(md,model,opt_fn=optim.Adam)

learner.model

ThreeCharsRNNAdd(
  (emb): Embedding(174, 87)
  (w_in): Linear(in_features=87, out_features=256, bias=True)
  (w_hidden): Linear(in_features=256, out_features=256, bias=True)
  (w_out): Linear(in_features=256, out_features=174, bias=True)
)

In [100]:
learner.fit(1e-2, 1, wds=1e-4, cycle_len=3)

epoch      trn_loss   val_loss                                
    0      1.811868   2.026871  
    1      1.647477   0.615167                                
    2      1.623399   0.820811                                 



[array([0.82081])]

## Generate text from samples

In [77]:
def next_char(model,inp):
    idxs = T(np.array([char2idx[c] for c in inp]))
    p = model(*V(idxs)) # (bs,n_uchars)
    i = np.argmax(to_np(p))
    return idx2char[i]

In [28]:
next_char(model,'tìn')

'h'

In [22]:
next_char(model,'Đan')

'g'

In [29]:
sampl = 'tìn'
for i in range(200):
    sampl+= next_char(model,sampl[-3:])
print(sampl)

tình nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Diệp đá nhỏ Di


Not good!

# n-char models - Interactive

In [9]:
class NCharsRNNAdd(nn.Module):
    def __init__(self,n_uchars,n_factors,n_hidden):
        super().__init__()
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_uchars,n_factors)
        self.w_in = nn.Linear(n_factors,n_hidden)
        self.w_hidden= nn.Linear(n_hidden,n_hidden)
        self.w_out = nn.Linear(n_hidden,n_uchars)
        
    def forward(self,*cs):
        h = V(torch.zeros((cs[0].size(0), self.n_hidden)).cuda())
        for c in cs:
            i_h = F.relu(self.w_in(self.emb(c)))
            h = h+i_h
            h = F.tanh(self.w_hidden(h))
        
        out = F.log_softmax(self.w_out(h)) # (bs,n_uchars)
        return out

class NCharsRNNConcat(nn.Module):
    def __init__(self,n_uchars,n_factors,n_hidden):
        super().__init__()
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_uchars,n_factors)
        self.w_in = nn.Linear(n_factors+n_hidden,n_hidden)
        self.w_hidden= nn.Linear(n_hidden,n_hidden)
        self.w_out = nn.Linear(n_hidden,n_uchars)
        
    def forward(self,*cs):
        h = V(torch.zeros((cs[0].size(0), self.n_hidden)).cuda())
        for c in cs:
            cat = torch.cat((h,self.emb(c)),1)
            i_h = F.relu(self.w_in(cat))
            h = F.tanh(self.w_hidden(i_h))
        
        out = F.log_softmax(self.w_out(h),dim=-1) # (bs,n_uchars)
        return out

In [10]:
l = len(text)
def get_chars_md(cs):
    X = []
    for i in range(0,cs):
        c_idxs = np.array([char2idx[c] for c in text[i:l-(cs-i)]])
#         print(c_idxs.shape)
        X.append(c_idxs)
    X = np.stack(X,axis=1)
    print(X.shape)
    y= np.array([char2idx[c] for c in text[cs:l]])
    print(y.shape)
    return X,y


In [98]:
cs=10
X,y = get_chars_md(cs)

(578167, 10)
(578167,)


In [99]:
n_vals=80000
val_idxs = np.arange(X.shape[0]-n_vals,X.shape[0])
md = ColumnarModelData.from_arrays('.', val_idxs, X, y, is_reg=False,bs=512,shuffle=False)

In [61]:
# check if data is loaded correctly
temp = next(iter(md.trn_dl))
len(temp)
''.join([idx2char[i] for i in temp[0][:50]])
''.join([idx2char[i] for i in temp[1][:50]])
''.join([idx2char[i] for i in temp[cs][:50]])

11

'Chương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc san'

'hương 1\nTiểu Long thức dậy trước tiên .\n\nLiếc sang'

'iểu Long thức dậy trước tiên .\n\nLiếc sang hai chiế'

In [62]:
temp = next(iter(md.val_dl))
len(temp)
''.join([idx2char[i] for i in temp[0][:50]])
''.join([idx2char[i] for i in temp[1][:50]])
''.join([idx2char[i] for i in temp[cs][:50]])

11

' tục vật nhau với nó, mày sẽ bị đo đất như tao chứ'

'tục vật nhau với nó, mày sẽ bị đo đất như tao chứ '

'hau với nó, mày sẽ bị đo đất như tao chứ gì?\n\nBất '

## Start training

### Model with hidden layers ADDED

In [89]:
# Model with hidden layers ADDED
n_factor = n_char//2
model = NCharsRNNAdd(n_char,n_factor,512).cuda()
model

NCharsRNNAdd(
  (emb): Embedding(174, 87)
  (w_in): Linear(in_features=87, out_features=512, bias=True)
  (w_hidden): Linear(in_features=512, out_features=512, bias=True)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

In [90]:
model = RNNModel(to_gpu(model))

learner = RNNLearner(md,model,opt_fn=optim.Adam)

learner.fit(1e-2, 1, wds=1e-4, cycle_len=4)

epoch      trn_loss   val_loss                              
    0      2.107023   3.711234  
    1      1.900071   2.745398                              
    2      1.713712   1.871645                              
    3      1.643395   1.705535                              


[array([1.70553])]

In [75]:
# opt = optim.Adam(model.parameters(), 1e-2)

# fit(model, md, 1, opt, F.nll_loss)
# set_lrs(opt, 1e-3)
# fit(model, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.728831   1.80382   



[array([1.80382])]

### Generate text from sample

In [76]:
text[2000:2500]

'ư đá .\n\nQuý ròm chép miệng :\n\n- Đầu mày cứng như vậy hèn gì học hoài không nhét vô lấy một chữ !\n\nLời châm chọc của Quý ròm làm Tiểu Long cụt hứng :\n\n- Đừng chơi quê anh em, mày !\n\nQuý ròm nhe răng cười :\n\n- Chứ mày luyện môn này chi vậy ?\n\n- Sao lại chi vậy ? - Tiểu Long hào hứng giải thích - Luyện môn này, đứa nào đánh tao, tao không cần đánh lại, chỉ cần đưa đầu ra đỡ là ...\n\n- ... "Rắc" một cái, cánh tay địch thủ gãy lìa ! - Quý ròm nhanh nhẩu tiếp lời .\n\nTiểu Long nhăn mặt :\n\n- Mày lúc nào '

In [91]:
next_char(model.model,'chỉ cần đư')

'ợ'

In [92]:
next_char(model.model,'Mày lúc nà')

'y'

In [94]:
sampl = 'chỉ cần đư'
for i in range(300):
    sampl+= next_char(model.model,sampl[-cs:])
print(sampl)

chỉ cần được mắt như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế này chuyện như thế 


### Model with hidden layer concatenated

In [100]:
# Model with hidden layers ADDED
n_factor = n_char//2
model = NCharsRNNConcat(n_char,n_factor,512).cuda()
model

model = RNNModel(to_gpu(model))

learner = RNNLearner(md,model,opt_fn=optim.Adam)

learner.fit(1e-2, 1, wds=1e-4, cycle_len=4)

NCharsRNNConcat(
  (emb): Embedding(174, 87)
  (w_in): Linear(in_features=599, out_features=512, bias=True)
  (w_hidden): Linear(in_features=512, out_features=512, bias=True)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

epoch      trn_loss   val_loss                              
    0      3.869167   9.664738  
    1      2.646773   3.560696                              
    2      2.056578   2.14612                               
    3      1.978247   1.987531                              


[array([1.98753])]

In [101]:
learner.fit(1e-3, 1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss                              
    0      1.919414   1.953072  
    1      1.901525   1.906739                              


[array([1.90674])]

In [184]:
# text generation
sampl = 'Mày lúc nà'
for i in range(300):
    sampl+= next_char(model.model,sampl[-cs:])
print(sampl)

Mày lúc nào chuyện gì thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như thế như 


# Char RNN from pytorch model

torch.nn.RNN:

For each element in the input sequence, each layer computes the following
function:


$$h_t = \tanh(w_{ih} * x_t + b_{ih}  +  w_{hh} * h_{(t-1)} + b_{hh})$$

where $h_t$ is the hidden state at time t, and $x_t$ is
the hidden state of the previous layer at time t-1  or the initial hidden state at time 0.

If nonlinearity='relu', then `ReLU` is used instead
of `tanh`.

In [187]:
cs=6
X,y = get_chars_md(cs)

n_vals=40000
val_idxs = np.arange(X.shape[0]-n_vals,X.shape[0])
md = ColumnarModelData.from_arrays('.', val_idxs, X, y, is_reg=False,bs=512,shuffle=False)

(578171, 6)
(578171,)


In [188]:
class CharRNN(nn.Module):
    def __init__(self,n_uchars,n_factors,n_hidden):
        super().__init__()
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_uchars,n_factors)
        
        # Replace these following 2 lines with pytorch RNN class
#         self.w_in = nn.Linear(n_factors,n_hidden)
#         self.w_hidden= nn.Linear(n_hidden,n_hidden)
        self.rnn = nn.RNN(n_factors,n_hidden)
        self.w_out = nn.Linear(n_hidden,n_uchars)
        
    def forward(self,*cs):
        h = V(torch.zeros((1, cs[0].size(0), self.n_hidden)).cuda())
#         for c in cs:
#             i_h = F.relu(self.w_in(self.emb(c)))
#             h = h+i_h
#             h = F.tanh(self.w_hidden(h))
        inp = self.emb(torch.stack(cs)) # 3d matrix, from (8 chars,bs) to (8,bs,n_fac)
        out,h = self.rnn(inp,h) 
        # outp will give back a growing matrix (all hidden states)
        #outp.size() will be (cs, bs, n_hidden)
        
        return F.log_softmax(self.w_out(out[-1]), dim=-1) # outp[-1] to grab the last tstate matrix(bs,n_uchars)

In [189]:
n_char

174

In [190]:
# Model with hidden layers ADDED
n_factor = n_char//2
model = CharRNN(n_char,n_factor,512).cuda()
model

CharRNN(
  (emb): Embedding(174, 87)
  (rnn): RNN(87, 512)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

In [143]:
opt = optim.Adam(model.parameters(), 1e-2)

fit(model, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(model, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.874065   2.058615  


[array([2.05861])]

epoch      trn_loss   val_loss                              
    0      1.582391   1.724492  


[array([1.72449])]

In [144]:
fit(model, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.551379   1.700512  
    1      1.527398   1.686372                              


[array([1.68637])]

### Using fast.ai learner class

In [191]:
model = RNNModel(to_gpu(model))

learner = RNNLearner(md,model,opt_fn=optim.Adam)

learner.fit(1e-2, 1, wds=1e-4, cycle_len=10)

epoch      trn_loss   val_loss                                
    0      2.377641   2.598731  
    1      2.266762   2.613416                                
    2      2.150789   2.18908                                 
    3      2.07715    2.130686                                
    4      1.98244    1.998532                                
    5      1.913213   1.95871                                 
    6      1.837456   1.872549                                
    7      1.787472   1.813219                                
    8      1.756168   1.781808                                
    9      1.763692   1.801813                                


[array([1.80181])]

### Check prediction

In [192]:
pred_log = learner.predict()
pred=np.argmax(pred_log,axis=1)

pred_text = ''.join([idx2char[idx] for idx in pred])

In [193]:
pred_text[:1000]

'c chhàn ciợng cai t chong chư chấ ca chhe càichong chn thn tum cài chch cuanhthm chom ca  cha cộnh  \nThỏng càn thy chông ca ca càn thou ch thhe chấy chưng cài choih ch cha cng chông  Thểug ciờn ci ch  ch  cuống chu tiy cội càc cột chư Hạn  \nTi càn thou ci thô  chong chm chả   càn thou cội cuan cào chư chong chhàc cangtuý ròm \n\n- Tồiciợc cai \n\nTồi ch chô chưc cài \n\n- Tay ciờ chấ chchcan ti caết chi cà chn tho  cai chn tii cai cho ci \n\nT T  \nTa cin nho ca gtà htit ciu  Thhy chong càc ci cangtho cộốngtai càn thou caết ca  chưên cà thoyện thưng chông cin chy ci ch hti  cộ cộệng  Thư Hạnh cho ci  cột chưn tuanh \n\n- Ta can tà cai ca \n\n- T  \nT Thư can t ciu chấ \n\n- Tuocàm  \nT Thì can tin tiu cà ti choi cài cng can tiy cờ \nTàn thou cit ciu \n\n- T  Thưng chi chng cat tà chư cai \nT Tồi ch ch  chưncuốnciai c Tàychchcan thng chn tà ti  \nTiìy chông chn th ti chy tho cingtuý ròm cing chon cng cha càn thou cai càc chi chyngta ch g  \nTêc cat taa choế cho  chư Hạnh chn tho cho 

In [194]:
true_text = ''.join([idx2char[idx] for idx in md.val_ds.y])
true_text[:1000]

'ở ngoài đường hơn ở trong nhà thì ba nghĩ vợ chồng con nên xem lại cách quan tâm chăm sóc của mình.\n\nNhưng lần này không rõ ba Văn Châu có nghe thấy những lời trách cứ của ông không. Tiếng giày gõ cồm cộp xuống nền đất mỗi lúc một nhỏ dần.\n\nBa Văn Châu đi khỏi chừng năm phút, Văn Châu mới quay vào nhà trong ngoắc bọn Quý ròm:\n\n- Ra được rồi!\n\nRồi nó khẽ nhún vai:\n\n- Bây giờ thì các bạn đã biết tôi là con trai hay con gái rồi chứ gì?\n\n- Ừ.\n\nBa đứa trẻ bẽn lẽn gật đầu. Ngay trong lúc đó bọn trẻ muốn hỏi Văn Châu biết bao nhiêu là chuyện nhưng không đứa nào đủ cản đảm mở miệng. Nhỏ Hạnh chỉ đảo mắt nhìn quanh:\n\n- Ba bạn về rồi hả?\n\n- Ừ.\n\n- Nhà bạn ở đâu thế?\n\n- Xa lắm.\n\n- Thế bạn đến đây là để chơi với ông bạn đấy ư?\n\nVăn Châu gật đầu:\n\n- Ừ. Nhưng tôi cũng sắp về nhà rồi. - Rồi nó nói như xua đuổi - Mà các bạn cũng nên về đi.\n\nThấy không còn cớ gì nấn ná, bọn Quý ròm đành chào ông của Văn Châu rồi lục tục tuôn ra cổng.\n\nLúc sắp sửa chia tay, nhỏ Hạnh còn c

Except for characters' names,(sometime they are still wrong), this model is absolutely terrible at generating Vietnamese text

## generate text from sample

In [145]:

sampl = 'Tiểu Long'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

Tiểu Long nghe thằng nhóc cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho


In [146]:
sampl = ' .\n\nQuý R'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

 .

Quý Rò và như thế này thằng nhóc cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó cho nó


# Multi label output / LSTM / GRU

So far we have been running the model on some amount of chars and predicting the next one. We will repeat the process, but we run the model on n(th) characters and predict (n+1)th characters, i.e see 1st, predict 2nd then see 2nd, predict 3rd ...

## prepare data

In [58]:
idx = [char2idx[c] for c in text]
cs=10

c_dat = [idxs[i:i+cs] for i in range(len(idx)-cs)]
c_in_dat = c_dat[:-1]
c_out_dat = c_dat[1:]

X1 = np.stack(c_in_dat)
y1 = np.stack(c_out_dat)
X1.shape
y1.shape

(578166, 10)

(578166, 10)

In [27]:
X1[:5]
y1[:5]

array([[ 27,  56, 110, 108,  61,  55,   2,  13,   1,  43],
       [ 56, 110, 108,  61,  55,   2,  13,   1,  43,  57],
       [110, 108,  61,  55,   2,  13,   1,  43,  57, 133],
       [108,  61,  55,   2,  13,   1,  43,  57, 133,  68],
       [ 61,  55,   2,  13,   1,  43,  57, 133,  68,   2]])

array([[ 56, 110, 108,  61,  55,   2,  13,   1,  43,  57],
       [110, 108,  61,  55,   2,  13,   1,  43,  57, 133],
       [108,  61,  55,   2,  13,   1,  43,  57, 133,  68],
       [ 61,  55,   2,  13,   1,  43,  57, 133,  68,   2],
       [ 55,   2,  13,   1,  43,  57, 133,  68,   2,  35]])

In [140]:
# try non-overlap set of characters
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(text)-cs-1, cs)]

# labels: exact same thing, offset by 1
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

X2 = np.stack(c_in_dat)
y2 = np.stack(c_out_dat)
X2.shape
y2.shape

(57817, 10)

(57817, 10)

In [30]:
X2[:5,:]

array([[ 27,  56, 110, 108,  61,  55,   2,  13,   1,  43],
       [ 57, 133,  68,   2,  35,  62,  61,  55,   2,  67],
       [ 56, 159,  51,   2,  52, 120,  72,   2,  67,  65],
       [110, 149,  51,   2,  67,  57,  91,  61,   2,  11],
       [  1,   1,  35,  57, 130,  51,   2,  66,  49,  61]])

In [31]:
y2[:5,:]

array([[ 56, 110, 108,  61,  55,   2,  13,   1,  43,  57],
       [133,  68,   2,  35,  62,  61,  55,   2,  67,  56],
       [159,  51,   2,  52, 120,  72,   2,  67,  65, 110],
       [149,  51,   2,  67,  57,  91,  61,   2,  11,   1],
       [  1,  35,  57, 130,  51,   2,  66,  49,  61,  55]])

In [76]:
class CharSeqRNN(nn.Module):
    def __init__(self,n_uchars,n_factors,n_hidden):
        super().__init__()
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_uchars,n_factors)

        self.rnn = nn.RNN(n_factors,n_hidden)
        self.w_out = nn.Linear(n_hidden,n_uchars)
        
    def forward(self,*cs):
        h = V(torch.zeros((1, cs[0].size(0), self.n_hidden)).cuda())

        inp = self.emb(torch.stack(cs)) # 3d matrix, from (8 chars,bs) to (8,bs,n_fac)
        out,h = self.rnn(inp,h) 
        # outp will give back a growing matrix (all hidden states)
        #outp.size() will be (cs, bs, n_hidden)
        
        return F.log_softmax(self.w_out(out), dim=-1) #(cs,bs,hidden) to (cs,bs,n_uchars)
    
# note that we use outp to get all n-char state matrices (cs,bs,n_hidden) instead of outp[-1], 
# and get output (blue arrow) for each state matrix h and get negative log loss for each

In [77]:
# loss function for multiple output RNN
def nll_loss_seq(inp, targ):
    #sl is cs (i.e 10)
    #bs is batch size
    #n_uchars is number of unique character
    #inp size is (cs,bs,n_uchars)

    sl,bs,n_uchars = inp.size() 
    #targ size (yt) is (bs,cs)
    #'transpose': change targ to (cs,bs),then flatten it to (cs x bs)
    # add 'contiguous' to make sure it really tranposes.
    targ = targ.transpose(0,1).contiguous().view(-1)
    
    #change inp to (cs x bs,n_uchars)
    return F.nll_loss(inp.view(-1,n_uchars), targ)

In [78]:
n_char

# Model with hidden layers ADDED
n_factor = n_char//2
model = CharSeqRNN(n_char,n_factor,512).cuda()
model

opt = optim.Adam(model.parameters(), 1e-2)

174

CharSeqRNN(
  (emb): Embedding(174, 87)
  (rnn): RNN(87, 512)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

# Overlap chars data model

In [79]:
n_vals=50000
val_idxs = np.arange(X1.shape[0]-n_vals,X1.shape[0])
md = ColumnarModelData.from_arrays('.', val_idxs, X1, y1, is_reg=False,bs=512,shuffle=False)

In [80]:
fit(model, md, 1, opt, nll_loss_seq)

epoch      trn_loss   val_loss                                
    0      1.926155   1.979435  



[array([1.97943])]

In [81]:
set_lrs(opt, 1e-3)
fit(model, md, 10, opt, nll_loss_seq)

epoch      trn_loss   val_loss                                
    0      1.65945    1.717063  
    1      1.639788   1.698863                                
    2      1.62261    1.68468                                 
    3      1.606875   1.674842                                
    4      1.592945   1.666674                                
    5      1.581559   1.660839                                
    6      1.570719   1.655175                                
    7      1.560939   1.65209                                 
    8      1.552154   1.648335                                
    9      1.544639   1.645312                                



[array([1.64531])]

## Generating text

In [82]:
def next_char(model,inp):
    idxs = T(np.array([char2idx[c] for c in inp]))
    p = model(*V(idxs))[-1]
    i = np.argmax(to_np(p))
    return idx2char[i]

In [83]:
sampl = 'tụi nhỏ '
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

tụi nhỏ Hạnh chỉ thấy nó chỉ theo dõi ngay từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ t


In [84]:
sampl = 'tụi nhóc'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

tụi nhóc nhìn vào nhà Quý ròm nhún và chân theo dõi ngay từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ


In [85]:
sampl = 'Quý ròm'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

Quý ròm đã biết như thế nào cũng chẳng biết được nhỏ Hạnh chỉ thấy nó chỉ theo dõi ngay từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ 


In [86]:
sampl = 'Tiểu Long'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

Tiểu Long và Quý ròm nhún và chân theo dõi ngay từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ từ 


## Check prediction

In [133]:
md_i = iter(md.val_dl)

In [134]:
show=2
for batch in md_i:
    if show==0: break
    batch_pred = model(*V(batch[:cs]))[-1] # get 10th char prediction only 
    batch_pred = np.argmax(to_np(batch_pred),axis=1)
    print('\nPredict: ')
    print(''.join(idx2char[i] for i in batch_pred[:500]))
    
    truth = to_np(batch[cs])[:,-1] # match with 10th char pred above
    print('\nTrue: ')
    print(''.join(idx2char[i] for i in truth[:500]))
    print('-'*10)
    show-=1


Predict: 
uý ròm màp tức lhinta chỏi nhi ngn nừ niomhân nin thng caợt đhìo 

Nh nọ đangran nặn nheo dê la  nam cồt nên nà phía sìc niợng 
Nóưng cừa đhe tau ta  nin tuý ròm mửnt mgiy tâ g cạn thô nhấy Văn Châu ring nonlắ lhi  tể chẳng niohàp  thườc nặt cáih ch g cẽc nh  cgaĩ  

Nừn Châu đing chan  chỉn nhưn tào nhn như chỏ Hhm nựu mh g thn thông thêng they nên tuý ròm  nóỏng câ sừy? nhế nưa ngo caa nàp nừn cáp thì h chì h tgư tộốn tàt ra đhỏi ning nhhc 

Nhông ti nảo ni  Qh bọ mọt tứt tat rầu têi  Qhấy thì

True: 
uý ròm lập tức nhô ra khỏi chỗ nấp và ba chân bốn cẳng rượt theo.

Cả ba rón rén men theo bờ rao rậm rạp lần về phía góc đường. Nhưng vừa thò đầu ra, bọn Quý ròm suýt nhảy dựng lên khi thấy Văn Châu đứng lù lù cách đó chừng ba mét, trước một cánh cổng sắt cao nghễu.

Văn Châu đang kiễng chân nhìn vào căn nhà nhỏ nằm sau cổng nên không trông thấy bọn Quý ròm, nhưng dù vậy, tim đứa nào đứa nấy vẫn đập thình thịch như muốn vọt ra khỏi đang ngực.

Không ai bảo ai, cả ba hấp tấp r

# Nonoverlap char data models

In [158]:
model = CharSeqRNN(n_char,n_factor,512).cuda()
model

opt = optim.Adam(model.parameters(), 1e-3)

CharSeqRNN(
  (emb): Embedding(174, 87)
  (rnn): RNN(87, 512)
  (w_out): Linear(in_features=512, out_features=174, bias=True)
)

In [159]:
X2.shape
y2.shape

(57817, 10)

(57817, 10)

In [161]:
n_vals=8000
val_idxs = np.arange(X2.shape[0]-n_vals,X2.shape[0])
md = ColumnarModelData.from_arrays('.', val_idxs, X2, y2, is_reg=False,bs=512,shuffle=False)

In [162]:
fit(model, md, 2, opt, nll_loss_seq)

epoch      trn_loss   val_loss                            
    0      2.405729   2.149568  
    1      2.043233   1.977288                            



[array([1.97729])]

In [163]:
set_lrs(opt, 1e-3)
fit(model, md, 10, opt, nll_loss_seq)

epoch      trn_loss   val_loss                            
    0      1.883453   1.90987   
    1      1.829624   1.870148                            
    2      1.786403   1.844538                            
    3      1.752143   1.825393                            
    4      1.723863   1.809057                            
    5      1.699202   1.795843                            
    6      1.676934   1.785109                            
    7      1.656508   1.776136                            
    8      1.637628   1.768822                            
    9      1.619993   1.763129                            



[array([1.76313])]

In [169]:
fit(model, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss                            
    0      1.600664   1.758776  
    1      1.586842   1.755373                            
    2      1.572191   1.752746                            
    3      1.557761   1.750838                            
    4      1.543799   1.749506                            



[array([1.74951])]

In [165]:
## Generating text

sampl = 'tụi nhỏ '
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

tụi nhỏ Hạnh nhau thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như t


In [166]:
sampl = 'tụi nhóc'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

tụi nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc nh


In [167]:
sampl = 'Quý ròm'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

Quý ròm đã thấy nhỏ Diệp chẳng biết thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì t


In [168]:
sampl = 'Tiểu Long'
for i in range(300):
    sampl+= next_char(model,sampl[-cs:])
print(sampl)

Tiểu Long chẳng biết thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế này thì thằng nhóc như thế
